In [213]:
# Load required libaries + save as a new file to keep original template in tact
import pandas as pd
import os
import json
import sys
import time
import hmac
import requests
from ftx_functions import get_price

In [214]:
# All FTX Futures Markets: https://ftx.com/markets
# 2 Coins Used for Application: BTC & ETH

In [215]:
# Pulls TODAY'S SPOT prices for BTC & ETH from FTX API
btc_spot = get_price("BTC/USD")
eth_spot = get_price("ETH/USD")

Pulling Today's Price for BTC/USD...
Pulling Today's Price for ETH/USD...


In [216]:
# Combines the SPOT price data into a single DataFrame and reformats for application composability
spot_prices = pd.concat([btc_spot, eth_spot], axis="rows", join="inner")
spot_prices = spot_prices.rename(columns={"Current Price":"spotPrice"})
spot_prices = spot_prices.rename_axis('symbol')
spot_prices["exchange"] = ["FTX", "FTX"]
spot_prices["underlying"] = ["BTC", "ETH"]
spot_prices = spot_prices.reset_index()
spot_prices = spot_prices[["exchange", "underlying", "symbol", "spotPrice"]]
spot_prices = spot_prices.set_index("underlying")

print("cryptoSpotPrices DataFrame")
display(spot_prices)


cryptoSpotPrices DataFrame


,exchange,symbol,spotPrice
underlying,,,
BTC,FTX,BTC/USD,43636.0
ETH,FTX,ETH/USD,3097.6


In [217]:
# Pulls TODAY's FUTURES prices for BTC & ETH from FTX API
btc_futures_mar = get_price("BTC-0325")
btc_futures_jun = get_price("BTC-0624")
eth_futures_mar = get_price("ETH-0325")
eth_futures_jun = get_price("ETH-0624")

Pulling Today's Price for BTC-0325...
Pulling Today's Price for BTC-0624...
Pulling Today's Price for ETH-0325...
Pulling Today's Price for ETH-0624...


In [218]:
# Combines the FUTURES price data into a single DataFrame
futures_prices = pd.concat([btc_futures_mar, btc_futures_jun,eth_futures_mar, eth_futures_jun], axis="rows", join="inner")

# Reformats DataFrame for application composability
futures_prices = futures_prices.rename_axis('symbol')
futures_prices["exchange"] = ["FTX", "FTX", "FTX", "FTX"]
futures_prices["underlying"] = ["BTC", "BTC", "ETH", "ETH"]
futures_prices["expiration"] = ["2022-03-25", "2022-06-24", "2022-03-25", "2022-06-24"]
futures_prices["expiration"]= pd.to_datetime(futures_prices['expiration'])
futures_prices = futures_prices.reset_index()
futures_prices = futures_prices[["exchange", "underlying", "symbol", "expiration", "Current Price"]]
futures_prices = futures_prices.set_index("underlying")
futures_prices["spotPrice"] = [spot_prices["spotPrice"]["BTC"], spot_prices["spotPrice"]["BTC"], spot_prices["spotPrice"]["ETH"], spot_prices["spotPrice"]["ETH"]]
futures_prices["annualizedReturn"] = (futures_prices["Current Price"] - futures_prices["spotPrice"]) / futures_prices["spotPrice"] * 12
futures_prices = futures_prices.drop(columns=['Current Price', 'spotPrice'])

print("cryptoFuturesPrices DataFrame")
futures_prices

cryptoFuturesPrices DataFrame


,exchange,symbol,expiration,annualizedReturn
underlying,,,,
BTC,FTX,BTC-0325,2022-03-25,0.02475
BTC,FTX,BTC-0624,2022-06-24,0.222752
ETH,FTX,ETH-0325,2022-03-25,0.013171
ETH,FTX,ETH-0624,2022-06-24,0.173166
